In [22]:
from KGLE_Dataset import *
import torch
from torch import nn
from torch.utils.data import DataLoader

In [24]:
train_set = KGLE_Dataset("./data/processed_train.csv")
train_load = DataLoader(dataset=train_set,
                        batch_size=1024,
                        num_workers=4,
                        shuffle=True)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
model = nn.Sequential(
    nn.Linear(10, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 200),
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
).to(device)

In [25]:
lossfn = nn.MSELoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [28]:
def train(num_iter: int):
    for epoch in range(num_iter):
        for i, (entry, target) in enumerate(train_load):
            entry = entry.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            preds = model(entry)

            optimizer.zero_grad()
            loss = lossfn(preds, target)
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print(f"Iter: {i}, RMSE: {torch.sqrt(loss)}")

In [27]:
train(100)

Iter: 0, RMSE: 93844.9453125
Iter: 1000, RMSE: 496.5100402832031


c:\program files\python39\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([580])) that is different to the input size (torch.Size([580, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iter: 0, RMSE: 560.5195922851562


KeyboardInterrupt: 